# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 9 2023 10:03AM,257028,10,9136659,MedStone Pharma LLC,Released
1,Feb 9 2023 10:03AM,257028,10,9136660,MedStone Pharma LLC,Released
2,Feb 9 2023 10:03AM,257028,10,9136661,MedStone Pharma LLC,Released
3,Feb 9 2023 10:03AM,257028,10,9136662,MedStone Pharma LLC,Released
4,Feb 9 2023 10:03AM,257028,10,9136663,MedStone Pharma LLC,Released
5,Feb 9 2023 10:03AM,257028,10,9136664,MedStone Pharma LLC,Released
6,Feb 9 2023 10:03AM,257028,10,9136665,MedStone Pharma LLC,Released
7,Feb 9 2023 10:03AM,257028,10,9136667,MedStone Pharma LLC,Released
8,Feb 9 2023 10:03AM,257028,10,9136668,MedStone Pharma LLC,Released
9,Feb 9 2023 10:03AM,257028,10,9136669,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,257024,Released,6
38,257025,Released,1
39,257026,Released,2
40,257027,Released,1
41,257028,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257024,NaN,NaN,6.0
257025,NaN,NaN,1.0
257026,NaN,NaN,2.0
257027,NaN,NaN,1.0
257028,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256917,1.0,2.0,4.0
256931,0.0,0.0,1.0
256934,0.0,0.0,43.0
256938,1.0,0.0,1.0
256946,0.0,0.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256917,1,2,4
256931,0,0,1
256934,0,0,43
256938,1,0,1
256946,0,0,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256917,1,2,4
1,256931,0,0,1
2,256934,0,0,43
3,256938,1,0,1
4,256946,0,0,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256917,1,2,4
1,256931,,,1
2,256934,,,43
3,256938,1,,1
4,256946,,,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 9 2023 10:03AM,257028,10,MedStone Pharma LLC
11,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations"
12,Feb 9 2023 9:56AM,257026,10,Emerginnova
14,Feb 9 2023 9:54AM,257024,10,Bio-PRF
20,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc."
63,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc."
64,Feb 9 2023 8:58AM,257020,16,"SHL Pharma, LLC"
65,Feb 9 2023 8:55AM,257022,10,"Digital Brands, LLC"
76,Feb 9 2023 8:53AM,257021,12,"SD Head USA, LLC"
77,Feb 9 2023 8:32AM,257014,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 9 2023 10:03AM,257028,10,MedStone Pharma LLC,,,11
1,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",,,1
2,Feb 9 2023 9:56AM,257026,10,Emerginnova,,,2
3,Feb 9 2023 9:54AM,257024,10,Bio-PRF,,,6
4,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",,,43
5,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",,,1
6,Feb 9 2023 8:58AM,257020,16,"SHL Pharma, LLC",,1,
7,Feb 9 2023 8:55AM,257022,10,"Digital Brands, LLC",,,11
8,Feb 9 2023 8:53AM,257021,12,"SD Head USA, LLC",,,1
9,Feb 9 2023 8:32AM,257014,10,ISDIN Corporation,,,70


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 10:03AM,257028,10,MedStone Pharma LLC,11,,
1,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1,,
2,Feb 9 2023 9:56AM,257026,10,Emerginnova,2,,
3,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6,,
4,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43,,
5,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",1,,
6,Feb 9 2023 8:58AM,257020,16,"SHL Pharma, LLC",,1,
7,Feb 9 2023 8:55AM,257022,10,"Digital Brands, LLC",11,,
8,Feb 9 2023 8:53AM,257021,12,"SD Head USA, LLC",1,,
9,Feb 9 2023 8:32AM,257014,10,ISDIN Corporation,70,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 10:03AM,257028,10,MedStone Pharma LLC,11,,
1,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1,,
2,Feb 9 2023 9:56AM,257026,10,Emerginnova,2,,
3,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6,,
4,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 10:03AM,257028,10,MedStone Pharma LLC,11.0,NaN,NaN
1,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
2,Feb 9 2023 9:56AM,257026,10,Emerginnova,2.0,NaN,NaN
3,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6.0,NaN,NaN
4,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 10:03AM,257028,10,MedStone Pharma LLC,11.0,0.0,0.0
1,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
2,Feb 9 2023 9:56AM,257026,10,Emerginnova,2.0,0.0,0.0
3,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6.0,0.0,0.0
4,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3854880,197.0,3.0,1.0
12,514023,2.0,0.0,0.0
15,513865,44.0,0.0,0.0
16,257020,0.0,1.0,0.0
19,2569827,65.0,0.0,1.0
20,514034,2.0,0.0,0.0
21,1027946,3.0,1.0,0.0
22,513991,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3854880,197.0,3.0,1.0
1,12,514023,2.0,0.0,0.0
2,15,513865,44.0,0.0,0.0
3,16,257020,0.0,1.0,0.0
4,19,2569827,65.0,0.0,1.0
5,20,514034,2.0,0.0,0.0
6,21,1027946,3.0,1.0,0.0
7,22,513991,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,197.0,3.0,1.0
1,12,2.0,0.0,0.0
2,15,44.0,0.0,0.0
3,16,0.0,1.0,0.0
4,19,65.0,0.0,1.0
5,20,2.0,0.0,0.0
6,21,3.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,197.0
1,12,Released,2.0
2,15,Released,44.0
3,16,Released,0.0
4,19,Released,65.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Executing,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Released,197.0,2.0,44.0,0.0,65.0,2.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,Released,197.0,2.0,44.0,0.0,65.0,2.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,Released,197.0,2.0,44.0,0.0,65.0,2.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()